In [1]:
'''
class CF(object):
    def __init__(self, train_path, test_path, similarity_path):
        self.train_path=train_path
        self.test_path=test_path
        self.similarity_path=similarity_path
        
        self.w = {} #dict of weight, movie by movie, [less][large]
        self.r = {} #dict of rate, movie by user
        
        self.a_m_r=0 #overall ave movie rate
        self.u_m_r={} #user ave rate
        self.m_m_r={} #movie ave rate
                
        init_data()   

    def init_data():
        header=['user_id', 'item_id', 'rating', 'timestamp']
        train_data = pd.read_csv(self,train_path, sep=',', names=header)
        test_data = pd.read_csv(self.test_path, sep=',' , names=header)
        similarity_data = pd.read_csv(self.similarity_path, sep=',' , names=header)
        
        for line in train_data.itertuples():        
            if line[2] in self.r:
                self.r[line[2]].update({line[1]: line[3]})
            else:
                self.r.update({line[2]: {line[1]: line[3]}})
    
            if line[1] not in self.u_r:
                u_r.update({line[1]: []})
            u_r[line[1]].append(line[3])
    
            if line[2] not in self.m_r:
                m_r.update({line[2]: []})
            m_r[line[2]].append(line[3])
    
            a_r.append(line[3])
        
        self.a_m_r=sum(a_r)/len(a_r)
        for user, rates in u_r.items():
            self.u_m_r.update({user: sum(rates)/len(rates)})
        for movie, rates in m_r.items():
            self.m_m_r.update({movie: sum(rates)/len(rates)})

        for key1 in m_r:
            if key1 not in w:
                self.w.update({key1: {}})
            for key2 in m_r:
                if key2>key1:
                    self.w[key1].update({key2: 1})
'''

"\nclass CF(object):\n    def __init__(self, train_path, test_path, similarity_path):\n        self.train_path=train_path\n        self.test_path=test_path\n        self.similarity_path=similarity_path\n        \n        self.w = {} #dict of weight, movie by movie, [less][large]\n        self.r = {} #dict of rate, movie by user\n        \n        self.a_m_r=0 #overall ave movie rate\n        self.u_m_r={} #user ave rate\n        self.m_m_r={} #movie ave rate\n                \n        init_data()   \n\n    def init_data():\n        header=['user_id', 'item_id', 'rating', 'timestamp']\n        train_data = pd.read_csv(self,train_path, sep=',', names=header)\n        test_data = pd.read_csv(self.test_path, sep=',' , names=header)\n        similarity_data = pd.read_csv(self.similarity_path, sep=',' , names=header)\n        \n        for line in train_data.itertuples():        \n            if line[2] in self.r:\n                self.r[line[2]].update({line[1]: line[3]})\n            else:

In [2]:
import pandas as pd

In [13]:
header=['user_id', 'item_id', 'rating', 'timestamp']
train_data = pd.read_csv('train.csv', sep=',', names=header)
test_data = pd.read_csv('test.csv', sep=',' , names=header)
#header=['item_id1', 'item_id2', 'sim']
#similarity_data = pd.read_csv('similarity.csv', sep=',' , names=header)
header=['item_id1', 'item_id2', 'w']
w_data = pd.read_csv('w.csv', sep=',' , names=header)

In [4]:
w = {} #dict of weight, movie by movie
r = {} #dict of rate, movie by user
u_r={} #user rate
m_r={} #movie rate
a_r=[] #all rate

In [18]:
for line in train_data.itertuples():        
    if line[2] in r:
        r[line[2]].update({line[1]: line[3]})
    else:
        r.update({line[2]: {line[1]: line[3]}})
    
    if line[1] not in u_r:
        u_r.update({line[1]: []})
    u_r[line[1]].append(line[3])
    
    if line[2] not in m_r:
        m_r.update({line[2]: []})
    m_r[line[2]].append(line[3])
    
    a_r.append(line[3])
    
'''
for line in similarity_data.itertuples():
    if line[1] not in w:
        w.update({line[1]: {}})
    if line[2] not in w:
        w.update({line[2]: {}})
    w[line[1]].update({line[2]: line[3]})
    w[line[2]].update({line[1]: line[3]})
'''
for line in w_data.itertuples():
    if line[1] not in w:
        w.update({line[1]: {}})
    w[line[1]].update({line[2]: line[3]})

In [6]:
a_m_r=sum(a_r)/len(a_r)
u_m_r={} #user ave rate
m_m_r={} #movie ave rate
for user, rates in u_r.items():
    u_m_r.update({user: sum(rates)/len(rates)})
for movie, rates in m_r.items():
    m_m_r.update({movie: sum(rates)/len(rates)})

In [7]:
def get_similar(i, x, w, k):
    N=[]
    tmp=sorted(w[i].items(), key=lambda item:item[1], reverse=True)
    for j, rate in tmp: 
        if x in r[j]:
            N.append(j)
        if len(N)>=k:
            break
    return N

In [24]:
#training
n=20 #size of N, similar movies
episilon=0.1
step=0.01
loop=100
for l in range(loop):
    diff=0
    Jw=0
    for i in w:
        dJw={}
        for x, rix in r[i].items():
            N=get_similar(i, x, w, n)
            bix=a_m_r+(u_m_r[x]-a_m_r)+(m_m_r[i]-a_m_r)
            sumj=bix-rix
            for j in N:
                if j not in dJw:
                    dJw.update({j: 0})
                sumd=bix-rix
                bjx=a_m_r+(u_m_r[x]-a_m_r)+(m_m_r[j]-a_m_r)
                sumj=sumj+w[i][j]*(r[j][x]-bjx)
                for k in N:
                    bkx=a_m_r+(u_m_r[x]-a_m_r)+(m_m_r[k]-a_m_r)
                    sumd=sumd+w[i][k]*(r[k][x]-bkx)
                dJw[j]=dJw[j]+sumd*(r[j][x]-bjx)
                
            Jw=Jw+sumj**2
        for j in dJw:
            diff=diff+abs(step*dJw[j])
            w[i][j]=w[i][j]-step*dJw[j]
            
    print('No.', l, 'training successfully!')
    print('error','\t', Jw)
    print('diff','\t', diff)
    print()
    if diff <= episilon:
        break

No. 0 training successfully!
error 	 1.4017855856643916e+21
diff 	 12088381109.151226

No. 1 training successfully!
error 	 1.3577963697173645e+21
diff 	 11585893015.442682

No. 2 training successfully!
error 	 1.3565769260290256e+21
diff 	 11878430282.80852

No. 3 training successfully!
error 	 1.2801395993953543e+21
diff 	 9914379783.150126

No. 4 training successfully!
error 	 1.3125917274630793e+21
diff 	 10330538989.654705

No. 5 training successfully!
error 	 1.2911917371908945e+21
diff 	 10296842789.131044

No. 6 training successfully!
error 	 1.3014706516698753e+21
diff 	 10629316878.653032

No. 7 training successfully!
error 	 1.310730858043742e+21
diff 	 10863751147.86875

No. 8 training successfully!
error 	 1.3168982161521624e+21
diff 	 11277236801.072327

No. 9 training successfully!
error 	 1.3219151007476873e+21
diff 	 12121089920.079212

No. 10 training successfully!
error 	 1.2994890005255865e+21
diff 	 10685178171.597403

No. 11 training successfully!
error 	 1.274918

No. 94 training successfully!
error 	 1.0328107885080757e+21
diff 	 10757917701.736828

No. 95 training successfully!
error 	 1.0268681516489351e+21
diff 	 10463082425.098324

No. 96 training successfully!
error 	 1.0523545593394423e+21
diff 	 11326920392.59655

No. 97 training successfully!
error 	 1.0247489411826536e+21
diff 	 11552864795.14571

No. 98 training successfully!
error 	 1.0644978551957866e+21
diff 	 11758501568.140955

No. 99 training successfully!
error 	 1.0099074013879146e+21
diff 	 10213338995.0775



In [25]:
w_path='w3.csv'
wlist=[]
for m1, value in w.items():
    for m2, wij in value.items():
        wlist.append([m1, m2, wij])
print(wlist[0])
df = pd.DataFrame(wlist)
df.to_csv(w_path, index=False, header=False)
print('Saved successfully!')

[673, 2949, 0.2527718814038512]
Saved successfully!


In [26]:
#testing
n=20
for line in test_data.itertuples():        
    user=line[1]
    movie=line[2]
    rate=line[3]
    if movie not in w:
        continue
    pre=a_m_r+(u_m_r[user]-a_m_r)+(m_m_r[movie]-a_m_r)
    N=get_similar(movie, user, w, n)
    for j in N:
        bjx=a_m_r+(u_m_r[user]-a_m_r)+(m_m_r[j]-a_m_r)
        pre=pre+w[movie][j]*(r[j][user]-bjx)
        if pre>5:
            pre=5
        if pre<0:
            pre=0
    print(user, '\t', movie, '\t', rate, '\t', pre)

4 	 45 	 3.0 	 1.1196965762824664
4 	 47 	 2.0 	 1.9409510666293102
4 	 125 	 5.0 	 3.4647425149162485
4 	 235 	 2.0 	 2.510195013666368
4 	 265 	 5.0 	 1.970353984008115
4 	 296 	 1.0 	 0
4 	 345 	 4.0 	 3.413353958629064
4 	 457 	 5.0 	 1.1423047776871242
4 	 492 	 5.0 	 2.0580007308454866
4 	 593 	 5.0 	 0
4 	 595 	 3.0 	 2.0518180175665655
4 	 708 	 4.0 	 4.020879790567514
4 	 898 	 5.0 	 0.8409140982311081
4 	 908 	 5.0 	 2.924974608401555
4 	 919 	 5.0 	 4.548775037037898
4 	 1080 	 5.0 	 4.601287537801638
4 	 1183 	 1.0 	 2.3321287273853133
4 	 1198 	 3.0 	 4.951735572217199
4 	 1213 	 4.0 	 2.2710416105340534
4 	 1259 	 4.0 	 1.6817646055511792
4 	 1279 	 2.0 	 0
4 	 1291 	 4.0 	 2.452355967547719
4 	 1391 	 1.0 	 0.13319763807957674
4 	 1597 	 1.0 	 3.2081422514622817
4 	 1617 	 2.0 	 3.7511091298642016
4 	 1719 	 4.0 	 3.086186141915183
4 	 1834 	 5.0 	 1.5106899197563397
4 	 1892 	 2.0 	 3.2013823299737116
4 	 1916 	 1.0 	 0.4733105334341686
4 	 1947 	 5.0 	 2.63320013446652

21 	 1230 	 2.5 	 4.488529062545877
21 	 1270 	 5.0 	 2.350117998019645
21 	 1391 	 0.5 	 3.8649321712826303
21 	 2011 	 5.0 	 4.950555140877156
21 	 2115 	 4.0 	 4.5206136375057016
21 	 2174 	 1.0 	 4.661190498575199
21 	 2376 	 4.5 	 4.254354425798779
21 	 2402 	 3.0 	 4.422167385861277
21 	 2700 	 2.5 	 2.671517661977234
21 	 2947 	 4.5 	 5
21 	 2990 	 4.5 	 2.247176044657827
21 	 3022 	 4.0 	 5
21 	 3793 	 3.5 	 4.984817279977926
21 	 3948 	 3.5 	 3.816386872066292
21 	 4545 	 4.0 	 5
21 	 5218 	 4.0 	 5
21 	 5481 	 3.0 	 5
21 	 5796 	 2.0 	 5
21 	 6373 	 3.0 	 2.4054458783250126
21 	 7373 	 3.0 	 3.7536264904459107
21 	 7569 	 4.5 	 4.923403990800985
21 	 8972 	 4.0 	 5
21 	 30825 	 3.5 	 4.3309158845732965
21 	 31433 	 3.5 	 5
21 	 33615 	 4.0 	 5
21 	 33794 	 3.5 	 5
21 	 34162 	 3.0 	 4.879695038304757
21 	 45672 	 3.0 	 4.259252124768345
21 	 47997 	 5.0 	 5
21 	 49272 	 4.0 	 4.9615624273392145
21 	 50189 	 2.0 	 4.1287206787383885
21 	 51084 	 3.5 	 5
21 	 53322 	 4.5 	 4.06

42 	 4153 	 3.0 	 5
42 	 4322 	 4.0 	 3.580831952203147
42 	 4349 	 4.0 	 4.152901596212765
42 	 4477 	 2.0 	 4.932601709150352
45 	 50 	 5.0 	 5
45 	 107 	 4.0 	 5
45 	 112 	 4.0 	 5
45 	 150 	 5.0 	 4.1654781340425036
45 	 223 	 5.0 	 3.8951781104212504
45 	 296 	 5.0 	 5
45 	 317 	 3.0 	 5
45 	 356 	 5.0 	 5
45 	 364 	 5.0 	 4.9460886292055655
45 	 367 	 4.0 	 3.288512945741435
45 	 466 	 3.0 	 5
45 	 474 	 5.0 	 4.048671493487019
45 	 541 	 3.0 	 3.659192004406118
45 	 546 	 2.0 	 0.9892150157155053
45 	 586 	 3.0 	 3.698267604678905
45 	 590 	 4.0 	 4.043332717223935
45 	 595 	 5.0 	 4.358139065625341
45 	 639 	 2.0 	 5
45 	 733 	 4.0 	 2.788834538706166
45 	 765 	 4.0 	 4.997602726542711
45 	 852 	 4.0 	 4.524765829931283
45 	 919 	 3.0 	 1.789227629977878
45 	 1021 	 2.0 	 5
45 	 1080 	 4.0 	 5
45 	 1198 	 4.0 	 5
45 	 1246 	 5.0 	 5
45 	 1259 	 3.0 	 4.724808729496889
45 	 1270 	 3.5 	 4.710959393017582
45 	 1291 	 5.0 	 5
45 	 1371 	 3.0 	 4.443329334929734
45 	 1374 	 3.0 	 4

66 	 529 	 4.5 	 3.9145401085928455
66 	 555 	 3.5 	 0
66 	 586 	 3.0 	 4.80091760529955
66 	 592 	 4.0 	 3.9343201755584505
66 	 593 	 4.0 	 0
66 	 762 	 2.5 	 4.188985396339011
66 	 919 	 3.5 	 4.176056983748857
66 	 953 	 4.0 	 4.873642099059632
66 	 1004 	 3.0 	 4.998585657147715
66 	 1027 	 3.5 	 3.9892748441286994
66 	 1036 	 4.0 	 3.2846884018409535
66 	 1080 	 4.0 	 0.31068118963796587
66 	 1089 	 4.0 	 0.38492766554401764
66 	 1093 	 3.5 	 3.2039102588669492
66 	 1198 	 3.0 	 1.6150469780689871
66 	 1213 	 3.5 	 3.661231524397294
66 	 1214 	 4.0 	 4.149777718304329
66 	 1215 	 4.5 	 3.547746145581983
66 	 1216 	 4.5 	 5
66 	 1217 	 5.0 	 4.819490357579757
66 	 1224 	 4.5 	 3.355547642945418
66 	 1243 	 5.0 	 3.5118169733481945
66 	 1246 	 4.5 	 5
66 	 1259 	 3.5 	 4.492276999253021
66 	 1302 	 4.5 	 0.34368532839578186
66 	 1326 	 0.5 	 3.0183316091011716
66 	 1356 	 3.5 	 3.8465460880348177
66 	 1358 	 5.0 	 4.586255970770849
66 	 1407 	 4.5 	 3.813553593529143
66 	 1449 	 4.

95 	 39 	 4.0 	 0.6371828699350622
95 	 358 	 3.0 	 0
95 	 480 	 3.5 	 2.6114744717270226
95 	 541 	 5.0 	 4.565448646514465
95 	 1036 	 5.0 	 2.054608390704262
95 	 1089 	 5.0 	 4.233030041591647
95 	 1094 	 4.0 	 1.994844297386609
95 	 1175 	 5.0 	 4.347219792305962
95 	 1247 	 4.5 	 4.304845888226767
95 	 1270 	 5.0 	 4.385168353350716
95 	 1291 	 3.5 	 0.09252581294604598
95 	 1374 	 4.5 	 1.7756786653961498
95 	 1529 	 4.0 	 1.5042873550335016
95 	 1617 	 4.0 	 3.173748171506365
95 	 1653 	 5.0 	 3.4379957666780427
95 	 1921 	 4.5 	 4.001874661572476
95 	 2115 	 4.5 	 3.9661656757431887
95 	 2268 	 4.0 	 1.8358335278170548
95 	 2528 	 4.0 	 2.0732926790171975
95 	 3033 	 4.0 	 2.297028087958335
95 	 3070 	 3.0 	 4.756628793917104
95 	 3793 	 4.0 	 3.232580814372756
95 	 4270 	 3.0 	 2.755181566936375
95 	 4732 	 1.0 	 2.0579881080451106
95 	 5377 	 4.0 	 3.263894314829776
95 	 5810 	 4.0 	 0.3193937749000815
95 	 5812 	 4.0 	 2.4933186653764356
95 	 6288 	 4.0 	 0.2146476028919444

...

572 	 852 	 3.0 	 3.1675631987977724
572 	 908 	 5.0 	 4.315900716970693
572 	 953 	 5.0 	 1.8338898424349346
572 	 1061 	 5.0 	 0.4519141981289299
572 	 1129 	 1.0 	 1.3786832536661786
572 	 1214 	 4.0 	 0.21233421813226636
572 	 1220 	 2.0 	 1.0056112461135347
572 	 1250 	 5.0 	 1.8348743785543895
572 	 1259 	 4.0 	 2.332409702519527
572 	 1267 	 5.0 	 3.9374910994033474
572 	 1287 	 5.0 	 2.899956221265557
572 	 1356 	 4.0 	 0.0683586443263332
572 	 1663 	 4.0 	 0.8984466415458199
572 	 1693 	 3.0 	 2.253428240542078
572 	 1894 	 3.0 	 2.117340007870229
572 	 1917 	 3.0 	 0.20506694387951552
572 	 1954 	 4.0 	 0
572 	 2006 	 3.0 	 0
572 	 2145 	 2.0 	 1.600066484649719
572 	 2394 	 3.0 	 0
572 	 2433 	 3.0 	 0.2503321295619299
572 	 2539 	 4.0 	 2.1075561544600943
572 	 2670 	 5.0 	 0.04432643175828852
572 	 2724 	 4.0 	 0.21322477999606934
572 	 2788 	 5.0 	 1.8712900703285094
572 	 3037 	 5.0 	 2.404120284326319
572 	 3072 	 4.0 	 0.7550836357519454
573 	 50 	 5.0 	 0
573 	 153 	 

592 	 356 	 5.0 	 0
592 	 364 	 5.0 	 4.871662703393188
592 	 415 	 1.0 	 4.05958104969164
592 	 435 	 3.0 	 4.164472963214873
592 	 457 	 5.0 	 0.6408909292328907
592 	 480 	 5.0 	 3.2706248381254337
592 	 500 	 5.0 	 4.870849232113202
592 	 533 	 4.0 	 4.750114302919664
592 	 540 	 3.0 	 4.732632757974036
592 	 587 	 4.0 	 2.620258504567169
592 	 588 	 5.0 	 2.3596989495003933
592 	 595 	 5.0 	 2.117286610330259
592 	 733 	 5.0 	 5
597 	 21 	 5.0 	 3.8805444151900925
597 	 34 	 4.0 	 4.784941895305714
597 	 39 	 4.0 	 1.8680262375916366
597 	 69 	 4.0 	 3.5732767356267976
597 	 125 	 4.0 	 1.697233952848881
597 	 223 	 3.0 	 0.7493838198270459
597 	 235 	 4.0 	 2.9289971515802424
597 	 246 	 5.0 	 2.88182950297828
597 	 380 	 2.0 	 0.9114777090511498
597 	 417 	 4.0 	 2.8385471143883514
597 	 457 	 5.0 	 1.0557790532826288
597 	 474 	 5.0 	 1.6684319775056873
597 	 480 	 5.0 	 0.6038220652352215
597 	 500 	 4.0 	 0
597 	 587 	 4.0 	 0.4600843477350999
597 	 592 	 4.0 	 4.610624209642

599 	 6850 	 2.0 	 4.301122701323514
599 	 6881 	 3.5 	 4.851234650766156
599 	 6890 	 3.0 	 5
599 	 6932 	 2.5 	 4.896491448704462
599 	 7022 	 4.0 	 4.196316658519684
599 	 7354 	 3.5 	 4.009843812681583
599 	 7371 	 2.5 	 4.879259043342194
599 	 7438 	 4.0 	 5
599 	 7982 	 3.5 	 5
599 	 8042 	 2.5 	 4.926534283575859
599 	 8169 	 3.0 	 4.7353102512991
599 	 8369 	 2.0 	 3.1146610016847958
599 	 8528 	 2.0 	 4.69300927851508
599 	 8529 	 2.0 	 5
599 	 8623 	 2.0 	 5
599 	 8807 	 2.5 	 5
599 	 8827 	 3.0 	 5
599 	 8874 	 3.0 	 4.742018942671675
599 	 8914 	 3.0 	 5
599 	 8961 	 2.5 	 5
599 	 9010 	 3.0 	 3.9083772013769282
599 	 27611 	 3.0 	 2.1494064106598785
599 	 27741 	 3.5 	 4.376809052529257
599 	 27773 	 4.0 	 5
599 	 27831 	 3.0 	 5
599 	 33794 	 3.0 	 4.653847027541775
599 	 34323 	 2.5 	 4.917354281437432
599 	 34332 	 2.0 	 3.821203872656468
599 	 36517 	 2.5 	 5
599 	 38886 	 2.5 	 4.867821083037033
599 	 39381 	 2.5 	 4.053181509683687
599 	 39414 	 2.5 	 3.9387828528660

600 	 7451 	 3.5 	 0
600 	 8169 	 3.5 	 4.456306227114856
600 	 8633 	 4.0 	 4.466923959912439
600 	 8874 	 4.5 	 3.8510907996526873
600 	 8949 	 3.5 	 3.7573603765008423
600 	 30825 	 3.0 	 0
600 	 30894 	 3.0 	 4.801147214057595
600 	 33794 	 2.5 	 2.4245365738274645
600 	 38886 	 4.5 	 2.738552253025279
600 	 39444 	 1.0 	 2.7620920492544863
600 	 48783 	 1.0 	 4.028738973785379
600 	 49530 	 1.0 	 3.793014454813455
600 	 54286 	 1.0 	 5
600 	 56775 	 0.5 	 0.49665731822775083
600 	 59725 	 2.5 	 1.717447349675043
601 	 47 	 4.0 	 5
601 	 50 	 5.0 	 5
601 	 1198 	 4.0 	 5
601 	 1917 	 4.0 	 0.015295400451779222
601 	 4306 	 4.0 	 5
601 	 33794 	 4.5 	 3.4069261856398887
601 	 48780 	 4.5 	 3.4971762355294733
601 	 50872 	 5.0 	 1.6369697717969116
601 	 60069 	 5.0 	 2.5515809285643267
601 	 72378 	 4.0 	 0
601 	 80549 	 4.0 	 0
601 	 81845 	 4.0 	 2.0368217534216884
601 	 89745 	 4.5 	 1.4429469278184972
601 	 95167 	 4.0 	 1.090242226192189
601 	 122918 	 4.0 	 0.2240254919732651
6

605 	 54190 	 4.0 	 5
605 	 55872 	 3.0 	 4.75537628574986
605 	 56152 	 3.5 	 2.6823030743116916
605 	 58347 	 4.5 	 4.993642131049781
605 	 59501 	 4.0 	 0
605 	 60937 	 2.5 	 1.88661938303901
605 	 63853 	 3.0 	 4.365413773289314
605 	 65685 	 3.0 	 4.2340756363221965
605 	 74530 	 3.0 	 3.8107624410625878
607 	 34 	 3.0 	 4.674470511601491
607 	 153 	 3.0 	 5
607 	 296 	 3.0 	 0.7531685905397376
607 	 318 	 5.0 	 0
607 	 423 	 3.0 	 4.771581367046183
607 	 480 	 4.0 	 3.8829293598862167
607 	 517 	 4.0 	 4.379669278358187
607 	 592 	 3.0 	 4.475176442271474
607 	 593 	 5.0 	 0
607 	 798 	 5.0 	 2.4945293482727253
607 	 949 	 4.0 	 0.19056398006838604
607 	 1036 	 5.0 	 4.421270186979013
607 	 1084 	 4.0 	 4.47498618430029
607 	 1094 	 3.0 	 4.600167431314579
607 	 1213 	 4.0 	 4.389757760057745
607 	 1214 	 3.0 	 4.979826175560347
607 	 1259 	 5.0 	 5
607 	 1270 	 3.0 	 4.883718057234718
607 	 1291 	 4.0 	 5
607 	 1327 	 5.0 	 2.305609905835431
607 	 1339 	 5.0 	 4.990014249953163


In [28]:
#prediction
n=20
prediction=[]
def predict(i, x, w, n):
    pre=a_m_r+(u_m_r[x]-a_m_r)+(m_m_r[i]-a_m_r)
    N=get_similar(i, x, w, n)
    for j in N:
        bjx=a_m_r+(u_m_r[x]-a_m_r)+(m_m_r[j]-a_m_r)
        pre=pre+w[i][j]*(r[j][x]-bjx)
    if pre>5:
        pre=5
    if pre<0:
        pre=0
    return pre

for user in u_r:
    for movie in m_r:
        if movie not in w:
            continue
        pre=predict(movie, user, w, n)
        prediction.append([user, movie, pre])
print(prediction[:5])

[[1, 1, 0], [1, 3, 4.001295085523713], [1, 6, 4.000221525474087], [1, 47, 5], [1, 50, 0]]


In [29]:
pre_path='pre.csv'
df = pd.DataFrame(prediction)
df.to_csv(pre_path, index=False, header=False)